In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('dark_background')

RK integration of coupled ODEs

In [ ]:
def dydx(x,y):
    #set the derivatives
    #our equation d^2y/dx^2=-y
    #dy/dx=z
    #dz/dx=-y
    #set y[0]=y
    #set y[1]=z
    
    #declare an array
    y_derivs = np.zeros(2)
    #set dydx=z
    y_derivs[0] = y[1]
    #set dzdx=-y
    y_derivs[1] = -1*y[0]
    return y_derivs

Define the 4th order RK

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):

    #declare k arrays
    #dydx is function of derivatives
    #xi is the value of x at step i
    #yi is the array of variables at step i
    #nv is ...

    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)

    C = [0, 1 / 5, 3 / 10, 3 / 5, 1, 7 / 8]
    A = [[0, 0, 0, 0, 0], [1 / 5, 0, 0, 0, 0], [3 / 40, 9 / 40, 0, 0, 0],
     [3 / 10, -9 / 10, 6 / 5, 0, 0], [-11 / 54, 5 / 2, -70 / 27, 35 / 27, 0],
     [1631 / 55296, 175 / 512, 575 / 13824, 44275 / 110592, 253 / 4096]]
    B = [37 / 378, 0, 250 / 621, 125 / 594, 0, 512 / 1771]
    B2 = [2825 / 27648, 0, 18575 / 48384, 13525 / 55296, 277 / 14336, 1 / 4]

    #define x at 1/2 step
    x_ipoh = xi + 0.5 * h
    #define x at 1 step
    x_ipo = xi + h
    #declare a temp y array
    y_temp = np.zeros(nv)
    #get k1 values
    y_derivs = dydx(xi, yi)
    k1[:] = h * y_derivs[:]

    #get k2 values
    y_temp[:] = yi[:] + (A[1][0]) * k1[:]
    y_derivs = dydx(xi + (C[1]) * h, y_temp)
    k2[:] = h * y_derivs[:]

    #get k3 values
    y_temp[:] = yi[:] + (A[2][0]) * k1[:] + (A[2][1]) * k2[:]
    y_derivs = dydx(xi + (C[2]) * h, y_temp)
    k3[:] = h * y_derivs[:]

    #get k4 values
    y_temp[:] = yi[:] + (A[3][0]) * k1[:] + (A[3][1]) * k2[:] + (
        A[3][2]) * k3[:]
    y_derivs = dydx(xi + (C[3]) * h, y_temp)
    k4[:] = h * y_derivs[:]

    #k5
    y_temp[:] = yi[:] + (A[4][0]) * k1[:] + (A[4][1]) * k2[:] + (
        A[4][2]) * k3[:] + (A[4][3]) * k4[:]
    y_derivs = dydx(xi + (C[4]) * h, y_temp)
    k5[:] = h * y_derivs[:]

    #k6
    y_temp[:] = yi[:] + (A[5][0]) * k1[:] + (A[5][1]) * k2[:] + (
        A[5][2]) * k3[:] + (A[5][3]) * k4[:] + (A[5][4]) * k5[:]
    y_derivs = dydx(xi + (C[5]) * h, y_temp)
    k6[:] = h * y_derivs[:]

    #advance y[:] by a step h
    yipo = yi + B[0]*k1[:] + B[1]*k2[:] +B[2]*k3[:] +B[3]*k4[:] +B[5]*k6[:] 
    ysipo = B2[0]*k1[:] + B2[1]*k2[:] +B2[2]*k3[:] +B2[3]*k4[:] +B2[5]*k6[:] 

    return yipo

Define an adaptive step size driver for RK4

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    #define a safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a max number of iterations
    imax = 10000
    #set an iteration variable
    i = 0
    #create an error
    Delta = np.full(nv, 2*tol)
    #remember the step
    h_step = h
    #adjust step
    while(Delta.max()/tol > 1.0):
        #estimate our error by taking one step of size h vs. two steps of size 1/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step) #one big step
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step) #two small steps
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        #if the error is too large, take a smaller step
        if (Delta.max()/tol > 1.0):
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        #check iteration
        if (i >= imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i= ",i)
        #iterate
        i +=1
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    return y_2, h_new, h_step

Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx, a, b, y_a, tol):
    #a is lower bound
    #b is upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    #an initial step size, very small
    h = 1.0e-4 * (b-a)
    #set max iters
    imax = 10000
    #set an iteration variable
    i = 0
    #set the number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    #set flag
    flag = 1
    #loop until on the right side
    while(flag):
        #calc y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        #update the step
        h = h_new
        #prevent an overshoot
        if (xi+h_step>b):
            #take a smaller step
            h = b-xi
            #recalc y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        #add the steps to the arrays 
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations 
        if (i >= imax):
            print("Max iterations reached.")
            raise StopIteration("Iteration number = ",i)
        #iterate
        i += 1
        #output some info
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i ,xi, h_step, b)
        print(s)
        #break if new xi == b
        if(xi == b):
            flag = 0
    return x,y
        

Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi
y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2
tolerance = 1.0e-6

x,y = rk4_mv(dydx, a, b, y_0, tolerance)

Plot the result

In [ ]:
plt.plot(x, y[:,0], 'o', label = 'y(x)')
plt.plot(x, y[:,1], 'o', label = 'dy/dx(x)')
xx = np.linspace(0, 2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label = 'sin(x)')
plt.plot(xx, np.cos(xx), label = 'cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

Calculate Error

In [ ]:
y_err = (y[:,0]-np.sin(x))
dydx_err = (y[:,1]-np.cos(x))
plt.plot(x, y_err, label = "y(x) error")
plt.plot(x, dydx_err, label = "dydx(x) error")
plt.legend(frameon = False)